Importing databases

In [1]:
!pip3 install pandas
import pandas as pd

Nov = pd.read_csv("data/November2025_5640.csv")
Oct = pd.read_csv("data/October2025_5640.csv")
Sep = pd.read_csv("data/September2025_5640.csv")

Sep

Defaulting to user installation because normal site-packages is not writeable


,Posted Date,Reference Number,Payee,Address,Amount
0,09/27/2025,24116415269493005085729,CLIPPER TRANSIT FARE SAN FRANCISCOCA,SAN FRANCISCO CA,-2.40
1,09/27/2025,24116415269493005057157,CLIPPER TRANSIT FARE SAN FRANCISCOCA,SAN FRANCISCO CA,-2.40
2,09/27/2025,24137465269100234562044,TST* VERVE COFFEE ROASTERSAN FRANCISCOCA,SAN FRANCISCO CA,-7.35
3,09/26/2025,24116415268491844151703,CLIPPER TRANSIT FARE SAN FRANCISCOCA,SAN FRANCISCO CA,-7.10
4,09/26/2025,24116415268491844050236,CLIPPER TRANSIT FARE SAN FRANCISCOCA,SAN FRANCISCO CA,-5.20
...,...,...,...,...,...
99,09/01/2025,24291285243000000115859,VALENCIA WHOLE FOODS SAN FRANCISCOCA,SAN FRANCISCO CA,-22.15
100,09/01/2025,24445005242100164650895,CLIPPER SYSTEMS MOBILE #1CONCORD CA,CONCORD CA,-3.00
101,08/30/2025,24692165241102388677400,SEPHORA POWELL SAN FRANCISCOCA,SAN FRANCISCO CA,-133.61
102,08/29/2025,24692165240101830039780,SQ *T&O SAN FRANCISCOCA,SAN FRANCISCO CA,-6.09


In [2]:
td = pd.concat([Nov, Oct])


td.columns = td.columns.str.lower().str.strip()



In [3]:
import re

def clean_merchant(name: str) -> str:
    name = name.lower()
    name = re.sub(r"\d+", "", name)
    name = re.sub(r"s\s#", "", name)
    name = re.sub(r"#", "", name)
    
    for city in ["san francisco", "berkeley", "ca", "oakland"]:
        name = name.replace(city, "")
    
    for n in ["pos", "transaction", "counter", "purchase", "terminal", "card"]:
        name = name.replace(n, "")

    return name.strip()

td["merchant_clean"] = td["payee"].apply(clean_merchant)



In [4]:
td.head()

,posted date,reference number,payee,address,amount,merchant_clean
0,11/26/2025,33086000000398179296708,TRAVEL CREDIT,NaN,91.17,travel credit
1,11/25/2025,32920401050084112972750,PAYMENT FROM CHK 8893 CONF#1fxp0hij7,NaN,150.00,payment from chk conffxphij
2,11/25/2025,24011345329100037420716,DEPOP WWW.DEPOP.COMNY,WWW.DEPOP.COM NY,-17.35,depop www.depop.comny
3,11/24/2025,24692165327106158743778,SEPHORA CAPE COD HYANNIS MA,HYANNIS MA,-43.57,sephora pe cod hyannis ma
4,11/24/2025,24011345326100110720381,DEPOP WWW.DEPOP.COMNY,WWW.DEPOP.COM NY,-22.72,depop www.depop.comny


In [5]:
unique_merchants = td["merchant_clean"].unique().tolist()
unique_merchants[:20]

['travel credit',
 'payment from chk  conffxphij',
 'depop www.depop.comny',
 'sephora pe cod hyannis ma',
 'corepower yoga - dubce',
 'city of mbridge -- nc',
 'sq *cida coffee bar mbridge ma',
 'sq *triangle coffee watertown ma',
 'payment from sav  confgchmqufry',
 'tst* verve coffee roaster',
 'amazon mktpl*betz amzn.com/billwa',
 'evergreen market',
 'nijiya market',
 'walgreen',
 "it's m time",
 'valencia whole foods',
 'ls ndy doll beauty -',
 'lyft *baywheels bike lyft.com',
 'sq *ritual coffee roaster',
 'spotify -- ny']

In [6]:
def guess_category(merchant: str):
    if any(word in merchant for word in [ "coffee", "roaster", "elaichi", "ritual", "verve", "cafe",
        "blue bottle", "flywheel", "arizmendi", "saint frank", "sidewalk"]):
        return "coffee"
    if any(word in merchant for word in [ "market", "whole foods", "bi-rite", "trader joe", "produce",
        "woori", "golden natural", "evergreen", "deli"]):
        return "groceries"
    if any(word in merchant for word in ["lyft", "uber", "baywheels", "clipper"]):
        return "transportation"
    if any(word in merchant for word in ["classpass", "corepower", "yoga"]):
        return "fitness"
    if any(word in merchant for word in ["spotify", "patreon"]):
        return "subscriptions"
    if any(word in merchant for word in ["li po", "liquor", "bar", "lounge", "tavern"]):
        return "drinks/going out"
    if any(word in merchant for word in ["salon", "beauty", "sephora", "nail", "scent", "perfume",
        "credo", "ls candy"]):
        return "beauty/personal care"
    if any(word in merchant for word in ["taqueria", "poke", "udon", "ramen", "dumpling", "burma",
        "boogaloos", "panchitas", "palmetto", "sweetgreen", "grubhub.com"]):
        return "eating out"
    if any(word in merchant for word in ["depop", "baggu", "sfmoma", "half price", "maido", "store", "good vibrations"]):
        return "shopping"
    if any(word in merchant for word in ["blick", "stationery", "photo", "hickory clay", "tiat.place",
        "needles & pens", "tattoo"]):
        return "hobbies_creative"
    if any(word in merchant for word in ["payment from", "sav", "chk"]):
        return "transfers"
        
    return "other"
    


In [7]:
suggested = {m: guess_category(m) for m in unique_merchants}
suggested

from collections import defaultdict


df_merchants = pd.DataFrame({
    "merchant": list(suggested.keys()),
    "category": list(suggested.values())
})

df_merchants

,merchant,category
0,travel credit,other
1,payment from chk conffxphij,transfers
2,depop www.depop.comny,shopping
3,sephora pe cod hyannis ma,beauty/personal care
4,corepower yoga - dubce,fitness
...,...,...
113,good vibrations - valenci,shopping
114,payment from sav conforiwcro,transfers
115,grubhub*lersros grubhub.com ny,eating out
116,ctlp*photo-mati,hobbies_creative


In [13]:
category_keywords["coffee"]
category_keywords["groceries"]

['evergreen market',
 'nijiya market',
 'valencia whole foods',
 'trader joe',
 'rheas market deli',
 'golden natural foods',
 "gus's community market",
 'horsies market',
 'international produce ma',
 'bi-rite market',
 'guerrero market and deli',
 'woori food market']

In [14]:
def classify_from_dict(merchant):
    for cat, keywords in category_keywords.items():
        if merchant in keywords:
            return cat
    return "other"

td["category"] = td["merchant_clean"].apply(classify_from_dict)
        

Training model <3

In [15]:
###%pip install scikit-learn

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

df_merchants["merchant_clean"] = df_merchants["merchant"].str.lower()

X = df_merchants["merchant_clean"]
y = df_merchants["category"]

model = Pipeline([
    ("tfidf", TfidfVectorizer(
        stop_words="english",
        ngram_range=(1, 2),   # better for names like "ritual coffee"
        min_df=1
    )),
    ("clf", LogisticRegression(max_iter=300))
])

model.fit(X, y)


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(ngram_range=(1, 2), stop_words='english')),
                ('clf', LogisticRegression(max_iter=300))])

In [16]:
def hybrid_classify(merchant: str):
    merchant = merchant.lower()
    cat = guess_category(merchant)
    if cat != "other":
        return cat
    ml_cat = model.predict([merchant])[0]
    return ml_cat
    
category_keywords = defaultdict(list)

for merchant in unique_merchants:
    cat = hybrid_classify(merchant)
    category_keywords[cat].append(merchant)

vectorizer = model.named_steps["tfidf"]
merchant_vec = vectorizer.transform([merchant])

# find nearest neighbors
similarities = (vectorizer.transform(df_merchants["merchant_clean"]) @ merchant_vec.T).toarray()
